In [2]:
import requests
import json
import pandas as pd
from config import api_key

In [3]:
filepath = "Resources/Item_Codes.csv"
item_codes = pd.read_csv(filepath)
item_codes.head()

,SUB CATEGORY CODE,ITEM CODE,DESCRIPTION
0,ALCBEVG,ALCBEVG,Alcoholic beverages
1,APPAREL,APPAREL,Apparel and services
2,APPAREL,BOYS,"Apparel, Boys, 2 to 15"
3,APPAREL,FOOTWEAR,Footwear
4,APPAREL,GIRLS,"Apparel, Girls, 2 to 15"


In [4]:
seriesids = [[],[],[],[]]
item_desc = [[],[],[],[]]
setcount = 0
rowcount = 0
for index,row in item_codes.iterrows():
    if rowcount == 50:
        setcount = setcount + 1
        rowcount = 0
    seriesids[setcount].append(f'CXU{row["ITEM CODE"]}LB0101M')
    item_desc[setcount].append(row["DESCRIPTION"])
    rowcount = rowcount + 1 
    
item_desc

[['Alcoholic beverages',
  'Apparel and services',
  'Apparel, Boys, 2 to 15',
  'Footwear',
  'Apparel, Girls, 2 to 15',
  'Apparel, Children under 2',
  'Apparel, Men and boys',
  'Apparel, Men, 16 and over',
  'Other apparel products and services',
  'Apparel, Women and girls',
  'Apparel, Women, 16 and over',
  'Cash contributions',
  'Net change in total assets and liabilities',
  'Net change in total assets',
  'Net change in total liabilities',
  'Number of consumer units (in thousands)',
  'Education',
  'Entertainment: other supplies, equip., & services',
  'Entertainment',
  'Entertainment: fees and admissions',
  'Pets',
  'Pets, toys, and playground equipment',
  'Toys, hobbies, and playground equipment',
  'Audio and visual equipment and services',
  'Eggs',
  'Food prep. by consumer unit, out-of-town trips',
  'Meats, poultry, fish, and eggs',
  'Bakery products',
  'Beef',
  'Cereals and bakery products',
  'Cereals and cereal products',
  'Dairy products',
  'Fats and o

In [5]:
headers = {'Content-type': 'application/json'}
json_data = []
for i in range(setcount+1):
    data = json.dumps({"seriesid": seriesids[i],"startyear":"2010", "endyear":"2011","registrationkey":api_key})
    p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
    json_data.append(json.loads(p.text))
    json_data

In [35]:
json_data[1]["Results"]['series'][22]
# item_codes["YEAR"] = ""
# item_codes["VALUE"] = ""
# item_codes.loc[0,"VALUE"] = json_data[0]["Results"]['series'][0]['data'][0]['value']
# item_codes


{'seriesID': 'CXUTOYSLB0101M', 'data': []}

In [38]:
item_codes["YEAR"] = ""
item_codes["VALUE"] = ""
i = 0

for index,row in item_codes.iterrows():
    if index%50 == 0 and index != 0:
        i = i + 1
    try:
        row["YEAR"] = json_data[i]["Results"]['series'][index-i*50]['data'][0]['year']
        row["VALUE"] = json_data[i]["Results"]['series'][index-i*50]['data'][0]['value']
    except:
        row["YEAR"] = ''
        row["VALUE"] = ''
        print(f"Row {index} doesn't have data")
 
item_codes

Row 20 doesn't have data
Row 22 doesn't have data
Row 24 doesn't have data
Row 57 doesn't have data
Row 80 doesn't have data
Row 91 doesn't have data
Row 114 doesn't have data
Row 134 doesn't have data
Row 135 doesn't have data
Row 136 doesn't have data
Row 137 doesn't have data
Row 138 doesn't have data


,SUB CATEGORY CODE,ITEM CODE,DESCRIPTION,YEAR,VALUE
0,ALCBEVG,ALCBEVG,Alcoholic beverages,2011,456
1,APPAREL,APPAREL,Apparel and services,2011,1740
2,APPAREL,BOYS,"Apparel, Boys, 2 to 15",2011,80
3,APPAREL,FOOTWEAR,Footwear,2011,321
4,APPAREL,GIRLS,"Apparel, Girls, 2 to 15",2011,117
...,...,...,...,...,...
149,TRANS,USEDCARS,"Vehicle purchases: Cars and trucks, used",2011,1339
150,TRANS,VEHFINCH,Vehicle finance charges,2011,233
151,TRANS,VEHOTHXP,Other vehicle expenses,2011,2454
152,TRANS,VEHPURCH,Vehicle purchases (net outlay),2011,2669
